# Introduction
This originally came about through https://github.com/malariagen/parasite-ops/issues/41. After creating the relevant code for this (https://github.com/malariagen/parasite-ops/blob/master/work/41_ENA_accessions_for_803xGB4_cross/20181130_ENA_accessions_for_803xGB4_cross.ipynb) using my previous method of querying mlwh directly, I decided to see if I could recreate the results using FITS. This revealed some discrepancies (see Conclusions at end of this notebook).


In [1]:
%run ../setup.ipynb
import pymysql

python 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.13.3
scipy 0.19.1
pandas 0.22.0
numexpr 2.6.4
pysam 0.8.4
pysamstats 0.24.3
petlx 1.0.3
vcf 0.6.8
h5py 2.7.0
tables 3.4.2
zarr 2.2.0
scikit-allel 1.1.10


In [2]:
# Inputs
mlwh_missing_exceptions_fn = '/nfs/team112_internal/rp7/src/github/malariagen/SIMS/meta/mlwh/mlwh_missing_exceptions.txt'
mlwh_study_exceptions_fn = '/nfs/team112_internal/rp7/src/github/malariagen/SIMS/meta/mlwh/mlwh_study_exceptions.txt'
mlwh_sample_exceptions_fn = '/nfs/team112_internal/rp7/src/github/malariagen/SIMS/meta/mlwh/mlwh_sample_exceptions.txt'
mlwh_taxon_exceptions_fn = '/nfs/team112_internal/rp7/src/github/malariagen/SIMS/meta/mlwh/mlwh_taxon_exceptions.txt'
sequencescape_alfresco_study_mappings_fn='/nfs/team112_internal/rp7/src/github/malariagen/SIMS/meta/mlwh/sequencescape_alfresco_study_mappings.txt'

input_dir = 'placeholder'


# Find files for MalariaGEN Pf samples

In [3]:
def set_irods_name(row):
    # First calculate prefix using nonhuman dependent on id_run and species (taxon)
    # Then calculate tag string (empty string if no tag)
    # Then calculate file extension
    # Then concat these three strings
    if (
        (row['taxon_id'] in [7165, 7173, 30066, 62324]) & # Anopheles gambiae, arabiensis, merus and funestus respectively
        (row['id_run'] <= 6750)
    ):
        prefix = "%d_%d_nonhuman" % (row['id_run'], row['position'])
    elif (
        ~(row['taxon_id'] in [7165, 7173, 30066, 62324]) & # Anopheles gambiae, arabiensis, merus and funestus respectively
        (row['id_run'] <= 7100)
    ):
        prefix = "%d_%d_nonhuman" % (row['id_run'], row['position'])
    else:
        prefix = "%d_%d" % (row['id_run'], row['position'])
        
    if np.isnan(row['tag_index']):
        tag_string = ''
    else:
        tag_string = '#%d' % row['tag_index']
    
    if row['id_run'] <= 10300:
        file_extension = '.bam'
    else:
        file_extension = '.cram'
    
    irods_filename = prefix + tag_string + file_extension
    return(irods_filename)

def set_sample_id(row):
    if row['sample'] is None:
        return('DS_%s' % row['name'])
    else:
        return('DS_%s' % row['sample'])

def is_valid_alfresco_code(s):
    try: 
        int(s[0:4])
        return True
    except ValueError:
        return False

def create_build_manifest(
#     taxon_ids=[5833, 5855, 5858, 5821, 7165, 7173, 30066], # Pf, Pv, Pm, Pb, Ag, A. arabiensis, A. merus
    taxon_ids=[5833, 36329, 5847, 57267, 137071], # Pf, 3D7, V1, Dd2, HB3
    # 1089 is excluded as this is R&D samples. 1204 and 1176 are excluded as these are CP1 samples. 1175 is excluded as some Pf R&D samples were incorrectly tagged with this study. 1157 is a Pv study - there are two suspected Pf samples in this study, but we need further investigation, and possible study change both in ROMA and in Sequencescape/mlwh/ENA/iRODS before we can include
    studies_to_exclude = ['1089-R&amp;D-GB-ALCOCK', '1204-PF-GM-CP1', '1176-PF-KE-CP1', '1175-VO-KH-STLAURENT', '1157-PV-MULTI-PRICE'],
    miseq_runs_to_allow = [13809, 13810], # Two Miseq runs within 24 samples on each from study 1095-PF-TZ-ISHENGOMA that were included in Pf 6.0
    input_dir = input_dir,
    mlwh_missing_exceptions_fn = mlwh_missing_exceptions_fn,
    mlwh_study_exceptions_fn = mlwh_study_exceptions_fn,
    mlwh_sample_exceptions_fn = mlwh_sample_exceptions_fn,
    mlwh_taxon_exceptions_fn = mlwh_taxon_exceptions_fn,
    sequencescape_alfresco_study_mappings_fn = sequencescape_alfresco_study_mappings_fn,
    output_columns = [
        'path',
        'study',
        'sample',
        'lane',
        'reads',
        'paired',
        'irods_path',
        'sanger_sample_id',
        'taxon_id',
        'study_lims',
        'study_name',
        'id_run',
        'position',
        'tag_index',
        'qc_complete',
        'manual_qc',
        'description',
        'instrument_name',
        'instrument_model',
        'forward_read_length',
        'requested_insert_size_from',
        'requested_insert_size_to',
        'human_percent_mapped',
        'subtrack_filename',
        'subtrack_files_bytes',
        'ebi_run_acc',
        'ebi_sample_acc',
    ]
):
    """Create a DataFrame to be used as a build manifest.
    
    A build manifest here is a list of all the "lanelets" that need to be
    included in a build. The output will typically be written to a
    tab-delmited file, either to use as input to vr-pipe, or perhaps as
    input to a vr-track DB.

    Args:
        taxon_ids (list of int): The taxons of the build (P. falciparum is '5833',
            P. vivax is '5855').
        sequencescape_alfresco_study_mappings_fn (str): filename of
            mappings from sequencscape to alfresco study mappings

    Returns:
        pd.DataFrame: The build manifest.

    """
    
    # Read in taxon exceptions file
    df_mlwh_taxon_exceptions = pd.read_csv(mlwh_taxon_exceptions_fn, sep='\t', dtype={'tag_index': str, 'taxon_id': int}, index_col='irods_filename')
    df_mlwh_taxon_exceptions['tag_index'].fillna('', inplace=True)
    # Identify which samples in exceptions file match the taxon of current build, and which don't
    df_mlwh_exceptions_this_taxon = df_mlwh_taxon_exceptions.loc[df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids)]
    df_mlwh_exceptions_other_taxa = df_mlwh_taxon_exceptions.loc[~(df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids))]

    # Read in sample exceptions file
    df_mlwh_sample_exceptions = pd.read_csv(mlwh_sample_exceptions_fn, sep='\t', index_col='irods_filename')

    # Read in study exceptions file
    df_mlwh_study_exceptions = pd.read_csv(mlwh_study_exceptions_fn, sep='\t', index_col='irods_filename')

    # Read in missing exceptions file
    df_mlwh_missing_exceptions = pd.read_csv(mlwh_missing_exceptions_fn, sep='\t', index_col='irods_filename')
    df_mlwh_missing_exceptions['derivative_sample_id'] = 'DS_' + df_mlwh_missing_exceptions['sample_id']
    df_mlwh_missing_exceptions = df_mlwh_missing_exceptions.loc[
        df_mlwh_missing_exceptions['taxon_id'].isin(taxon_ids)
    ]

    # Read in SequenceScape-Alfresco study mappings
    df_sequencescape_alfresco_study_mappings = pd.read_csv(sequencescape_alfresco_study_mappings_fn, sep='\t', index_col='seqscape_study_id')
    sequencescape_alfresco_study_mappings_dict = df_sequencescape_alfresco_study_mappings.loc[
        :,
#         df_sequencescape_alfresco_study_mappings['build_flag']==1,
        'alfresco_study_code'
    ].to_dict()
    
    # Read in data from mlwh matching this taxon, plus samples from exceptions file matching this taxon
    conn = pymysql.connect(
        host='mlwh-db',
        user='mlwh_malaria',
        password='Solaris&2015',
        db='mlwarehouse',
        port=3435
    )
    
    sql_query = 'SELECT \
        study.name as study_name, \
        study.id_study_lims as study_lims, \
        sample.supplier_name as sample, \
        sample.name, \
        sample.sanger_sample_id, \
        sample.taxon_id, \
        iseq_product_metrics.id_run, \
        iseq_product_metrics.position, \
        iseq_product_metrics.tag_index, \
        iseq_product_metrics.num_reads, \
        iseq_product_metrics.human_percent_mapped, \
        iseq_run_lane_metrics.instrument_name, \
        iseq_run_lane_metrics.instrument_model, \
        iseq_run_lane_metrics.paired_read, \
        iseq_run_lane_metrics.qc_complete, \
        iseq_flowcell.manual_qc, \
        iseq_flowcell.requested_insert_size_from, \
        iseq_flowcell.requested_insert_size_to, \
        iseq_flowcell.forward_read_length, \
        iseq_run_status_dict.description \
    FROM \
        study, \
        iseq_flowcell, \
        sample, \
        iseq_product_metrics, \
        iseq_run_status, \
        iseq_run_lane_metrics, \
        iseq_run_status_dict \
    WHERE \
        study.id_study_tmp = iseq_flowcell.id_study_tmp and \
        iseq_flowcell.id_sample_tmp = sample.id_sample_tmp and \
        iseq_flowcell.manual_qc = 1 and \
        iseq_product_metrics.id_iseq_flowcell_tmp = iseq_flowcell.id_iseq_flowcell_tmp and \
        iseq_run_status.id_run = iseq_product_metrics.id_run and \
        iseq_product_metrics.id_run = iseq_run_lane_metrics.id_run and \
        iseq_product_metrics.position = iseq_run_lane_metrics.position and \
        iseq_run_status.iscurrent = 1 and \
        ( ( iseq_run_lane_metrics.instrument_model != "MiSeq" ) or (iseq_product_metrics.id_run in (%s)) ) and \
        iseq_run_status.id_run_status_dict = iseq_run_status_dict.id_run_status_dict and \
        study.faculty_sponsor = "Dominic Kwiatkowski" and \
        ( ( sample.taxon_id in (%s) ) or ' % (
            ', '.join([str(x) for x in miseq_runs_to_allow]),
            ', '.join([str(x) for x in taxon_ids])
        )
    sql_query = sql_query + ' or '.join(
        df_mlwh_taxon_exceptions.loc[df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids)].apply(
            lambda x: '(iseq_product_metrics.id_run="%s" and iseq_product_metrics.position="%s" and iseq_product_metrics.tag_index="%s")' % (x['id_run'], x['position'], x['tag_index']),
            1
        )
    )
    sql_query = sql_query + ')'
#         (iseq_flowcell.manual_qc = 1 or iseq_flowcell.manual_qc is null) and \
    
    df_return = pd.read_sql(sql_query, conn)

    # Replace missing taxon_id with -1 (can't have missing int values in pandas Series)
    df_return['taxon_id'] = df_return['taxon_id'].fillna(-1).astype('int32')

    # Determine file name in iRods
    df_return['irods_filename'] = df_return.apply(set_irods_name, 1)
    df_return.set_index('irods_filename', inplace=True)

    # Determine alfresco study and change any incorrect studies
    df_return['alfresco_study_code'] = df_return['study_lims'].apply(
        lambda x: sequencescape_alfresco_study_mappings_dict[int(x)] if int(x) in sequencescape_alfresco_study_mappings_dict else ''
    )
    empty_alfresco_study_code = (df_return['alfresco_study_code'] == '')
    df_return.loc[empty_alfresco_study_code, 'alfresco_study_code'] = df_return.loc[empty_alfresco_study_code, 'study_name']
    
    study_exception_indexes = df_mlwh_study_exceptions.index[
        df_mlwh_study_exceptions.index.isin(df_return.index)
    ]
    df_return.loc[study_exception_indexes, 'alfresco_study_code'] = df_mlwh_study_exceptions.loc[study_exception_indexes, 'alfresco_study_code']

    # Change any incorrect taxon IDs
    taxon_exception_indexes = df_mlwh_exceptions_this_taxon.index[
        df_mlwh_exceptions_this_taxon.index.to_series().isin(df_return.index)
    ]
    df_return.loc[taxon_exception_indexes, 'taxon_id'] = df_mlwh_exceptions_this_taxon.loc[taxon_exception_indexes, 'taxon_id']

    # Determine sample ID and change any incorrect sample IDs
    df_return['derivative_sample_id'] = df_return.apply(set_sample_id, 1)
    sample_exception_indexes = df_mlwh_sample_exceptions.index[
        df_mlwh_sample_exceptions.index.isin(df_return.index)
    ]    
    df_return.loc[sample_exception_indexes, 'derivative_sample_id'] = 'DS_' + df_mlwh_sample_exceptions.loc[sample_exception_indexes, 'sample_id']
    df_return.drop(['sample', 'name'], axis=1, inplace=True)
    
    # Remove any samples that are not in this taxon
    df_return = df_return.loc[~df_return.index.isin(df_mlwh_exceptions_other_taxa.index)]
    
    # Merge in missing exceptions
    df_return = df_return.append(df_mlwh_missing_exceptions.drop('study_group', 1))
    
    # Remove any samples that don't have an alfresco study
    invalid_codes = ~df_return['alfresco_study_code'].apply(is_valid_alfresco_code)
    if(np.count_nonzero(invalid_codes) > 0):
        print('Removing %d files with invalid alfresco_study_code:' % np.count_nonzero(invalid_codes))
        print(df_return['alfresco_study_code'].loc[invalid_codes].value_counts())
        df_return = df_return.loc[~invalid_codes]

    # Determine filename in subtrack (for very old samples, there is only a .srf file in subtrack)
    # Note we determined the id_run cutoff (5750) for bam/srf after looking at previous runs. There seems
    # to be a grey area between runs 5500 and 5750 where some files are srf and some are bam
    df_return['subtrack_filename'] = df_return.index
    df_return.loc[df_return['id_run'] < 5750, 'subtrack_filename'] = df_return.loc[df_return['id_run'] < 5750, 'subtrack_filename'].apply(lambda x: x.replace('.bam', '.srf'))
    
    # Read in data from substrack, most importantly to get run accessions
    sql_query = "\
    SELECT \
        files.file_name as irods_filename, \
        files.bytes as subtrack_files_bytes, \
        files.timestamp as subtrack_files_timestamp, \
        files.public_date as subtrack_files_public_date, \
        submission.id as subtrack_submission_id, \
        submission.created as subtrack_submission_created, \
        submission.release_date as subtrack_submission_release_date, \
        submission.timestamp as subtrack_submission_timestamp, \
        submission.ext_db as subtrack_submission_ext_db, \
        submission.ebi_sample_acc, \
        submission.ebi_exp_acc, \
        submission.ebi_study_acc, \
        submission.ebi_sub_acc, \
        submission.ebi_run_acc \
    FROM \
        submission, \
        files \
    WHERE \
        files.sub_id = submission.id AND \
        files.file_name in (%s)\
    " % ("'" + "', '".join(df_return['subtrack_filename']) + "'")

    conn= pymysql.connect(
        host='shap',
        user='ega_dataset',
        password='ega_dataset',
        db='subtrack',
        port=3303
    )

    df_run_accessions = pd.read_sql(sql_query, conn).set_index('irods_filename')
    
    # Merge in subtrack data
    df_return = df_return.join(df_run_accessions, on='subtrack_filename', how='left')

    # Remove unwanted studies
    df_return = df_return.loc[~df_return['alfresco_study_code'].isin(studies_to_exclude)]

    # Create other columns
    df_return.rename(columns={'alfresco_study_code': 'study', 'paired_read': 'paired'}, inplace=True)
    df_return['lane'] = df_return.index.to_series().apply(lambda x: x.split('.')[0])
    df_return['path'] = df_return.apply(lambda x: "%s/%s" % (input_dir, x.name), 1)
    df_return['reads'] = df_return['num_reads'].fillna(-1).astype(int)
    df_return['irods_path'] = df_return.apply(lambda x: "/seq/%d/%s" % (x['id_run'], x.name), 1)
    df_return['sample'] = df_return['derivative_sample_id'].str.replace('DS_', '')

    # Remove lanes with zero reads
    df_return = df_return.loc[df_return['reads'] != 0]
    
    # Remove control samples (see emails from Sonia 03/05/2018 12:20 and from Richard 09/10/2018 13:26)
#     df_return = df_return.loc[df_return['sample'].str.slice(0, 1) != 'C']
    df_return = df_return.loc[(df_return['sample'].str.slice(0, 1) != 'C') & (df_return['sample'] != 'control')]
    
    # Restrict to certain columns
    if output_columns is not None:
        df_return = (
            df_return[output_columns]
            .sort_values(['study', 'sample', 'id_run', 'position', 'tag_index'])
        )
    
    print(df_return.shape)

    return(df_return) 


In [4]:
df_assay_data = create_build_manifest()

Removing 63 files with invalid alfresco_study_code:
AT-rich amplification                                 53
Test sequencing of high level human contamination.     6
Plasmodium falciparum genome variation 1               4
Name: alfresco_study_code, dtype: int64


/nfs/users/nfs_r/rp7/miniconda3/envs/biipy/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (3170, "Memory capacity of 8388608 bytes for 'range_optimizer_max_mem_size' exceeded. Range optimization was not done for this query.")
  result = self._query(query)


(18130, 27)


# Create file of 1126-PF-LAB-FAIRHURST samples

In [5]:
df_1126 = (
    df_assay_data
    .loc[df_assay_data['study'] == '1126-PF-LAB-FAIRHURST']
    [['sample', 'ebi_run_acc', 'ebi_sample_acc']]
)
print(df_1126.shape)
df_1126[0:3]

(60, 3)


,sample,ebi_run_acc,ebi_sample_acc
irods_filename,,,
12876_2#9.cram,PG0050-CX2,ERR570014,ERS427908
15795_3#21.cram,PG0106-Cx,ERR905471,ERS639326
12876_2#1.cram,PG0443-C,ERR570006,ERS427900


# Attempt to do the same with FITS

In [6]:
conn = pymysql.connect(
    host='malloc-db',
    user='solaris_ro',
    db='mm6_fits',
    port=3309
)

sql_query = '''
SELECT # The main SELECT statement
    # List of columns to return for each file
    (SELECT full_path FROM file WHERE file.id=sample2file.file_id) AS file_full_path, # The full path of the file
    (SELECT `value` FROM sample2tag WHERE sample2file.sample_id=sample2tag.sample_id AND tag_id=3594 LIMIT 1) AS `study_name`, # Sequenscape study name
    (SELECT value FROM sample2tag WHERE sample2tag.sample_id=sample2file.sample_id AND tag_id=3561 LIMIT 1) AS oxford_code, # The Oxford code of the sample; note the "LIMIT 1" to ensure only one value!
    (SELECT `value` FROM file2tag WHERE sample2file.file_id=file2tag.file_id AND tag_id=3602 LIMIT 1) AS `ebi_run_acc`,
    (SELECT `value` FROM file2tag WHERE sample2file.file_id=file2tag.file_id AND tag_id=3587 LIMIT 1) AS `ebi_sample_acc`
FROM sample2file # The "backbone table" of the query
WHERE sample_id IN (SELECT DISTINCT sample_id FROM sample2tag WHERE tag_id=3600 AND value IN (5833,36329,5847,57267,137071)) # Get all P falciparum samples
AND NOT EXISTS (SELECT * FROM file WHERE file.id=file_id AND (full_path LIKE "%phix%" OR full_path LIKE "%\_human%") ) # Exclude phiX/human
AND NOT EXISTS (SELECT * FROM file2tag WHERE sample2file.file_id=file2tag.file_id AND tag_id=8 AND `value`='0') # Exclude zero-size files, where file size is set
AND file_id IN (SELECT file_id FROM file2tag WHERE tag_id=3576 AND value IN ('bam','cram')) # Only BAM or CRAM
AND file_id NOT IN (SELECT parent FROM file_relation WHERE relation=3595) # Only one of multiple data files (e.g. BAM and CRAM with identical data)
AND file_id NOT IN (SELECT DISTINCT file_id FROM file2tag WHERE tag_id=3592 AND `value`='GBS') # Exclude genotyping-by-sequencing files
AND file_id NOT IN (SELECT DISTINCT file_id FROM file2tag WHERE tag_id=3577 AND `value`='0') # Exclude files with zero reads
AND file_id NOT IN (SELECT DISTINCT file_id FROM file2tag WHERE tag_id=3581 AND `value`='0') # Exclude manual_qc 0
HAVING
    `study_name` LIKE "%1126%"
;
'''

df_fits = pd.read_sql(sql_query, conn)
df_fits['irods_filename'] = df_fits['file_full_path'].apply(lambda x: x.split('/')[-1])
df_fits.set_index('irods_filename', inplace=True)
print(df_fits.shape)
df_fits[0:3]

(68, 5)


,file_full_path,study_name,oxford_code,ebi_run_acc,ebi_sample_acc
irods_filename,,,,,
12876_2#3.cram,/seq/12876/12876_2#3.cram,ILB_1126-PF-LAB-FAIRHURST,PG0466-C,ERR570008,None
13566_3#19.cram,/seq/13566/13566_3#19.cram,ILB_1126-PF-LAB-FAIRHURST,PG0453-C,ERR656298,None
15795_3#19.cram,/seq/15795/15795_3#19.cram,ILB_1126-PF-LAB-FAIRHURST,PG0509-C,ERR905469,None


In [7]:
# The following shows that 68 files have only 60 unqiue ox codes
len(df_fits['oxford_code'].unique())

60

# Sanity check we get the same information from both queries when irods_filename matches

In [8]:
df_join = df_1126.join(df_fits, rsuffix='_fits')
print(df_join.shape)
df_join[0:3]

(60, 8)


,sample,ebi_run_acc,ebi_sample_acc,file_full_path,study_name,oxford_code,ebi_run_acc_fits,ebi_sample_acc_fits
irods_filename,,,,,,,,
12876_2#9.cram,PG0050-CX2,ERR570014,ERS427908,/seq/12876/12876_2#9.cram,ILB_1126-PF-LAB-FAIRHURST,PG0050-CX2,ERR570014,None
15795_3#21.cram,PG0106-Cx,ERR905471,ERS639326,/seq/15795/15795_3#21.cram,ILB_1126-PF-LAB-FAIRHURST,PG0106-Cx,ERR905471,None
12876_2#1.cram,PG0443-C,ERR570006,ERS427900,/seq/12876/12876_2#1.cram,ILB_1126-PF-LAB-FAIRHURST,PG0443-C,ERR570006,None


In [9]:
df_join.loc[df_join['sample'] == df_join['oxford_code']].shape

(60, 8)

In [10]:
df_join.loc[df_join['ebi_run_acc'] == df_join['ebi_run_acc_fits']].shape

(60, 8)

### Interim conclusion
Where we have the same files, other metadata (sample ID and run accession) matches perfectly

# Determine why we have multiple files for some samples

In [11]:
df_fits.loc[df_fits['oxford_code'].duplicated(False)]

,file_full_path,study_name,oxford_code,ebi_run_acc,ebi_sample_acc
irods_filename,,,,,
12876_4#27.cram,/seq/12876/12876_4#27.cram,ILB_1126-PF-LAB-FAIRHURST,PG0469-C,ERR570032,None
12876_4#28.cram,/seq/12876/12876_4#28.cram,ILB_1126-PF-LAB-FAIRHURST,PG0469-C,ERR570033,None
12876_2#2.cram,/seq/12876/12876_2#2.cram,ILB_1126-PF-LAB-FAIRHURST,PG0454-C,ERR570007,None
12876_3#13.cram,/seq/12876/12876_3#13.cram,ILB_1126-PF-LAB-FAIRHURST,PG0454-C,ERR570018,None
12876_4#33.cram,/seq/12876/12876_4#33.cram,ILB_1126-PF-LAB-FAIRHURST,PG0446-C,ERR570038,None
12876_2#5.cram,/seq/12876/12876_2#5.cram,ILB_1126-PF-LAB-FAIRHURST,PG0446-C,ERR570010,None
12876_2#10.cram,/seq/12876/12876_2#10.cram,ILB_1126-PF-LAB-FAIRHURST,PG0455-C,ERR570015,None
12876_3#14.cram,/seq/12876/12876_3#14.cram,ILB_1126-PF-LAB-FAIRHURST,PG0455-C,ERR570019,None
12876_4#26.cram,/seq/12876/12876_4#26.cram,ILB_1126-PF-LAB-FAIRHURST,PG0457-C,ERR570031,None


In [12]:
', '.join(df_fits.loc[df_fits['oxford_code'].duplicated(False), 'oxford_code'].unique().tolist())

'PG0469-C, PG0454-C, PG0446-C, PG0455-C, PG0457-C, PG0470-C, PG0456-C, PG0445-C'

### Discrepancies for PG0469-C

In [13]:
!imeta ls -d /seq/12876/12876_4#27.cram | grep -A 1 'sample$'
!imeta ls -d /seq/12876/12876_4#27.cram | grep -A 1 'sample_supplier_name$'
!imeta ls -d /seq/12876/12876_4#27.cram | grep -A 1 'total_reads$'

attribute: sample
value: 3019STDY5769913
attribute: sample_supplier_name
value: PG0469-C
attribute: total_reads
value: 62354864


In [14]:
!imeta ls -d /seq/12876/12876_4#28.cram | grep -A 1 'sample$'
!imeta ls -d /seq/12876/12876_4#28.cram | grep -A 1 'sample_supplier_name$'
!imeta ls -d /seq/12876/12876_4#28.cram | grep -A 1 'total_reads$'

attribute: sample
value: 3019STDY5769913
attribute: total_reads
value: 90052


### Discrepancies for PG0445-C

In [15]:
!imeta ls -d /seq/12876/12876_4#25.cram | grep -A 1 'sample$'
!imeta ls -d /seq/12876/12876_4#25.cram | grep -A 1 'sample_supplier_name$'
!imeta ls -d /seq/12876/12876_4#25.cram | grep -A 1 'total_reads$'

attribute: sample
value: 3019STDY5769889
attribute: sample_supplier_name
value: PG0445-C
attribute: total_reads
value: 61025274


In [16]:
!imeta ls -d /seq/12876/12876_2#4.cram | grep -A 1 'sample$'
!imeta ls -d /seq/12876/12876_2#4.cram | grep -A 1 'sample_supplier_name$'
!imeta ls -d /seq/12876/12876_2#4.cram | grep -A 1 'total_reads$'

attribute: sample
value: 3019STDY5769889
attribute: total_reads
value: 174416


# Why doesn't my query against mlwh pull in these extra files?

In [17]:
conn = pymysql.connect(
    host='mlwh-db',
    user='mlwh_malaria',
    password='Solaris&2015',
    db='mlwarehouse',
    port=3435
)

In [18]:
pd.read_sql('SELECT id_iseq_flowcell_tmp FROM iseq_product_metrics WHERE id_run=12876 AND position=4 AND tag_index=27;', conn)

,id_iseq_flowcell_tmp
0,2924788


In [19]:
pd.read_sql('SELECT id_iseq_flowcell_tmp FROM iseq_product_metrics WHERE id_run=12876 AND position=4 AND tag_index=28;', conn)

,id_iseq_flowcell_tmp
0,None


# Conclusion
FITS contains two files for 8 samples in study 1126-PF-LAB-FAIRHURST (PG0469-C, PG0454-C, PG0446-C, PG0455-C, PG0457-C, PG0470-C, PG0456-C, PG0445-C). It seems that in each case, the two files both have the same sequencescape sample ID, but only one of the files has a sample_supplier_name attached. It also appears to be the case that the files without the sample_supplier_name have far fewer reads and as such are unlikely to be useful even if they are actually from the same sample. The reasons my query against mlwh is not pulling in the second files for these samples is that iseq_product_metrics.id_iseq_flowcell_tmp is not populated for them.

It is recommended that we do the following:
- Talk to Sonia about following up with sequencing core to understand why the second file was given the same sequencescape ID, but not a sample_supplier_name
- Determine how many other cases we have in FITS for which the supplier_sample_name is not populated
- Make a decision on what we should do with such samples, e.g.
    - Exclude entirely
    - Keep in FITS but mark in some way
    - Leave them as are
- Determine and document a way to implement the above decision
- Implement the decision

A second issue is that sample_accessions are not being populated with the query above run against FITS. I'll create a separate issue to track this.